##### Project: Opioid Exposed Infant Covariates
##### Investigator: Stephen Patrick, Sarah Loch
##### Programmers: Sander Su, Chris Guardo
##### Date Created: 01/17/23
##### Last Modified: 09/30/25

#### Notes:
This notebook depends on both the mom_hcv_hiv_lab and the mom_hcv_hiv_icd notebook, and performs an inner join to show which patients have both ICD and Lab values 


In [0]:
%run "../Project_modules"

In [0]:
phenotype_table_location = " ***Insert file location*** "
phenotype_table=spark.sql(f"SELECT * FROM {phenotype_table_location}")

##### Prepare the cohort

In [0]:
mom_exposed_opioid = get_mom_cohort(phenotype_table)
mom_exposed_opioid.createOrReplaceTempView("mom_exposed_opioid")

##### Combine cohort with covar outcomes

In [0]:
sql="""

  select
  distinct a.*,
  case
    when b.baby_person_id is not null then 1
    else 0
  end as maternal_hcv_lab_positive,
  case
    when c.baby_person_id is not null then 1
    else 0
  end as maternal_hcv_lab_quantitative,
  case
    when d.baby_person_id is not null then 1
    else 0
  end as maternal_hcv_icd,
  case
    when e.baby_person_id is not null then 1
    else 0
  end as maternal_hcv_icd_1ip2op,
  case
    when eip.baby_person_id is not null then 1
    else 0
  end as maternal_hcv_icd_1ip,
  case
    when eop.baby_person_id is not null then 1
    else 0
  end as maternal_hcv_icd_2op,
  case
    when f.baby_person_id is not null then 1
    else 0
  end as maternal_hiv_lab_positive,
  case
    when g.baby_person_id is not null then 1
    else 0
  end as maternal_hiv_lab_quantitative,
  case
    when i.baby_person_id is not null then 1
    else 0
  end as maternal_hiv_icd,
  case
    when j.baby_person_id is not null then 1
    else 0
  end as maternal_hiv_icd_1ip2op,
  case
    when jip.baby_person_id is not null then 1
    else 0
  end as maternal_hiv_icd_1ip,
  case
    when jop.baby_person_id is not null then 1
    else 0
  end as maternal_hiv_icd_2op
from
  mom_exposed_opioid a
  left join global_temp.hcv_hiv_lab_positive_prenatal_opioid_hcv b on a.mom_person_id = b.mom_person_id
  and a.baby_person_id = b.baby_person_id
  left join global_temp.hcv_hiv_lab_quantative_prenatal_opioid_hcv c on a.mom_person_id = c.mom_person_id
  and a.baby_person_id = c.baby_person_id
  left join (
    select
      *
    from
      global_temp.hcv_hiv_icd_prenatal
    where
      type = 'HCV'
  ) d on a.mom_person_id = d.mom_person_id
  and a.baby_person_id = d.baby_person_id
  left join global_temp.icd_prenatal_visit_hcv_ip1op2_update e on a.mom_person_id = e.mom_person_id
  and a.baby_person_id = e.baby_person_id
  left join global_temp.icd_prenatal_visit_hcv_ip1_update eip on a.mom_person_id = eip.mom_person_id
  and a.baby_person_id = eip.baby_person_id
  left join global_temp.icd_prenatal_visit_hcv_op2_update eop on a.mom_person_id = eop.mom_person_id
  and a.baby_person_id = eop.baby_person_id
  left join global_temp.hcv_hiv_lab_positive_prenatal_opioid_hiv f on a.mom_person_id = f.mom_person_id
  and a.baby_person_id = f.baby_person_id
  left join global_temp.hcv_hiv_lab_quantative_prenatal_opioid_hiv g on a.mom_person_id = g.mom_person_id
  and a.baby_person_id = g.baby_person_id
  left join (
    select
      *
    from
      global_temp.hcv_hiv_icd_prenatal
    where
      type = 'HIV'
  ) i on a.mom_person_id = i.mom_person_id
  and a.baby_person_id = i.baby_person_id
  left join global_temp.icd_prenatal_visit_hiv_ip1op2_update j on a.mom_person_id = j.mom_person_id
  and a.baby_person_id = j.baby_person_id
  left join global_temp.icd_prenatal_visit_hiv_ip1_update jip on a.mom_person_id = jip.mom_person_id
  and a.baby_person_id = jip.baby_person_id
  left join global_temp.icd_prenatal_visit_hiv_op2_update jop on a.mom_person_id = jop.mom_person_id
  and a.baby_person_id = jop.baby_person_id
  
  """

outcome_df=spark.sql(sql)
outcome_df.name='mom_hcv_hiv_output'
register_parquet_global_view(outcome_df)

In [0]:
df_inspection("global_temp.mom_hcv_hiv_output","all")


### Save Output for future use

In [0]:
outcome_df.write.mode("overwrite").saveAsTable(f"covariate_output.mom_hcv_hiv_output_all")